# Project 3 - The molecule forge

Dit is mijn jupyter notebook over mijn 3e project, in dit project ga ik werken met AI en proberen om nieuwe fictieve moleculen te maken

Tips voor mijzelf
- Numpy arrays voor speed?

### Het maken van een Virtual Environment
Voor coderen in python is het belangrijk om een Virtual Environment te maken, hiermee kun je per project een eigen python install hebben als het ware. 
Er zijn een aantal redenen om een Venv te gebruiken:
 - Scheiden van projecten, bijvoorbeeld bij het ene project heb je numpy versie 1.5 nodig en bij een ander project heb je versie 2 nodig. Doormiddel van Venv's kun je dit dan makkelijk scheiden.
 - Voorkomt vervuiling van je environment, er zijn geen onnodige packages die mee worden gestuurd als het project word geplubliceerd.
 - Een Venv is makkelijk te maken op een ander systeem waar de packages niet zijn geinstalleerd doormiddel van een requirements.txt.

Stukje voor imports 

In [1]:
from rdkit import Chem
import numpy as np

from itertools import chain
import random

Dit is een functie die ik zelf heb gemaakt om de eerst n aantal regels in een bestand te openen.

In [2]:
def read_lines(n):
    lines = []
    with open("guacamol_v1_train.smiles", "r") as file:
        for l in range(n):
            line = file.readline().strip()
            if not line:
                break
            lines.append(line)
    return lines

Stukje code om de eerste x aantal codes op te slaan en te printen naar de console.

In [3]:
molecules_amount = 100
molecules = read_lines(molecules_amount)
print(molecules)

['CCC(C)(C)Br', 'CCCN(CCc1cccc(-c2ccccc2)c1)C(=O)C1OC(C(=O)O)=CC(N)C1NC(C)=O', 'Oc1ccc(C2CC(c3ccccc3)=NN2C(=S)Nc2ccccc2)cc1', 'CC1(C)OCC2OC3(C4OC(C)(C)OC4CO)OC(C)(C)OC3C2O1', 'COC(=O)c1cc(C(=CCCCC(=O)SC)c2cc(Cl)c(OC)c(C(=O)OC)c2)cc(Cl)c1OC', 'Cc1cc(COc2ccc(NC(=O)C3CN(C)CCC3C(=O)NO)cc2)c2ccccc2n1', 'CCOC(=O)c1ccc(O)c(-n2cc3c(c2-c2ccccc2)c(=O)n(C)c(=O)n3C)c1', 'COc1cc(OC)c2c(-c3cccc(-c4ccc(C#N)cc4)c3)cc(=O)oc2c1', 'COc1cc2[nH]c(C(=O)Nc3ccc(F)cc3)cc2c(OC)c1OC', 'COc1cc(F)cc(-c2ccc(C(CC(=O)O)NC(=O)C3CCCN3S(=O)(=O)c3cc(Cl)cc(Cl)c3)cc2)c1', 'COc1cc(-c2cc(OC)c(-n3c(=O)ccc4cc(S(=O)(=O)Nc5cccnn5)ccc43)cc2F)ccc1Cl', 'CCCCC1(C)CC(CO)C(CCCC)(OC)OO1', 'CCCCCCCCCCCCNc1ccc2c3c(cccc13)C(=O)N(CCCN1CCCNCCNCCCNCC1)C2=O', 'COC(=O)C1=C(c2cc3ccccc3o2)CC2CCC1N2C(=O)NCc1cccc2ccccc12', 'Cc1ccccc1C=Cc1cccc(C(F)(F)P(=O)(O)O)c1', 'Nc1cc(C2CCNCC2)cc(NC2CCCC2)n1', 'CCN(C(C)=O)c1ccc(OC)c2nc(NC(=O)C3CCN(C(=O)c4cccc(C(F)(F)F)c4)CC3)sc12', 'COC(=O)C1CCC(C)C(c2ccc(C)cc2)N1C(=O)c1ccc(C=NOCC(O)COCc2ccco2)cc1', 'CCOC(=O)C1

Ook moet ik een functie hebben om te kijken of de gemaakte moleculen daadwerkelijk echt bestaan, hiervoor gebruik ik de functie MolFromSmiles(), deze functie pakt een van mijn smiles moleculen en splitst deze in losse atomen. De functie plakt deze dan na elkaar en probeert deze in mol formaat van Rdkit te zetten, mocht dit niet lukken dan is dit invalid en krijg ik None terug.

In [4]:
def is_valid_smiles(molecule):
    mol = Chem.MolFromSmiles(molecule)
    if mol is not None:
        return 'Valid'
    else:
       return 'Not valid'

In [5]:
random_molecule_num = random.randint(0, len(molecules) - 1)
random_molecule = molecules[random_molecule_num]

print(random_molecule)
print(is_valid_smiles(random_molecule))

wrong_molecule = random_molecule[0:-1]

print(wrong_molecule)
print(is_valid_smiles(wrong_molecule))

O=C(CN(Cc1ccco1)C(=O)c1ccc(Cl)cc1)Nc1ccc(Cl)c(Cl)c1
Valid
O=C(CN(Cc1ccco1)C(=O)c1ccc(Cl)cc1)Nc1ccc(Cl)c(Cl)c
Not valid


[16:07:28] SMILES Parse Error: unclosed ring for input: 'O=C(CN(Cc1ccco1)C(=O)c1ccc(Cl)cc1)Nc1ccc(Cl)c(Cl)c'


### Tokenization
Hieronder staat een functie die een array aan moleculen kan omzetten in losse characters om deze erna naar nummers om te zetten zodat mijn modellen deze nummers kunnen lezen en een predictie op maken.

In [6]:
def molecules_to_tokens(molecules):
    tokens = list()
    for molecule in molecules:
        tokens.append(list(molecule))
    return tokens

tokens = molecules_to_tokens(molecules[0:5])
print(tokens)

[['C', 'C', 'C', '(', 'C', ')', '(', 'C', ')', 'B', 'r'], ['C', 'C', 'C', 'N', '(', 'C', 'C', 'c', '1', 'c', 'c', 'c', 'c', '(', '-', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', '1', ')', 'C', '(', '=', 'O', ')', 'C', '1', 'O', 'C', '(', 'C', '(', '=', 'O', ')', 'O', ')', '=', 'C', 'C', '(', 'N', ')', 'C', '1', 'N', 'C', '(', 'C', ')', '=', 'O'], ['O', 'c', '1', 'c', 'c', 'c', '(', 'C', '2', 'C', 'C', '(', 'c', '3', 'c', 'c', 'c', 'c', 'c', '3', ')', '=', 'N', 'N', '2', 'C', '(', '=', 'S', ')', 'N', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', 'c', '1'], ['C', 'C', '1', '(', 'C', ')', 'O', 'C', 'C', '2', 'O', 'C', '3', '(', 'C', '4', 'O', 'C', '(', 'C', ')', '(', 'C', ')', 'O', 'C', '4', 'C', 'O', ')', 'O', 'C', '(', 'C', ')', '(', 'C', ')', 'O', 'C', '3', 'C', '2', 'O', '1'], ['C', 'O', 'C', '(', '=', 'O', ')', 'c', '1', 'c', 'c', '(', 'C', '(', '=', 'C', 'C', 'C', 'C', 'C', '(', '=', 'O', ')', 'S', 'C', ')', 'c', '2', 'c', 'c', '(', 'C', 'l', ')', 'c', '(', 'O', 'C', ')', 

Hieronder heb ik een functie geschreven die een set aan tokens pakt en hieruit alle unieke characters haalt, deze worden vervlgens omgezet naar een dictionary die er nummers aan koppelt.

In [7]:
def create_vocab(tokens):
    unique_tokens = set(chain.from_iterable(tokens))
    print(f"Unique tokens: {unique_tokens}")

    sorted_tokens = sorted(unique_tokens)
    print(f"Sorted tokens: {sorted_tokens}")

    char_to_index = {}
    for i, token in enumerate(sorted_tokens, start=1):
        char_to_index[token] = i
    print(f"Character to index mapping: {char_to_index}")
    return char_to_index

vocab = create_vocab(tokens)


Unique tokens: {'N', 'c', '-', '(', 'l', 'B', 'C', 'S', '1', '=', '3', '4', '2', ')', 'O', 'r'}
Sorted tokens: ['(', ')', '-', '1', '2', '3', '4', '=', 'B', 'C', 'N', 'O', 'S', 'c', 'l', 'r']
Character to index mapping: {'(': 1, ')': 2, '-': 3, '1': 4, '2': 5, '3': 6, '4': 7, '=': 8, 'B': 9, 'C': 10, 'N': 11, 'O': 12, 'S': 13, 'c': 14, 'l': 15, 'r': 16}


Nu dat ik een vocabulary heb gemaakt kan ik de moleculen die ik heb omgezet naar tokens veranderen in cijfers.

In [8]:
numeric_tokens = [[vocab[char] for char in molecule] for molecule in tokens]
print(numeric_tokens)

[[10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16], [10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12], [12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4], [10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4], [10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 14, 14, 1, 10, 15, 2, 14, 1, 12, 10, 2, 14, 1, 10, 1, 8, 12, 2, 12, 10, 2, 14, 5, 2, 14, 14, 1, 10, 15, 2, 14, 4, 12, 10]]


### Padding en sifting
Padding en spacing is nodig omdat een AI model elke input van de zelfde lengte moet hebben, een molecuul van bijvoorbeeld een lengte van 10 moet ook samen kunnen met een molecuul van bijvoorbeeld een lengte van 40. Ik doe dit door te kijken welk molecuul in de numeric array de langste is en bij de andere nullen toevoegen tot ik aan de zelfde lengte zit. Bij mijn toepassen zit wel nog een fout, als ik op de hele database ga trainen dan kan het zijn dat er grotere moleculen voorkomen en ik dus mijn model moet aanpassen.

Shifting is het toevoegen van een 0 aan het begin van een molecuul bijvoorbeeld een molecuul als C=O (in SMILES) is dan in tokens "C, =, O" en in nummers [3, 2, 4] doormiddel van shiften worden de nummers dan [0, 3, 2] en weet het model dat die de 0 moet gaan voorspellen, dit is hoe een RNN sequentialy kan leren.

Als ik shifting gebruik moet ik ook één bij de lengte toevoegen om rekening te houden met het extra getal.

In [9]:
def find_longest_molecule(molecules):
    longest = 0
    for n in range(len(molecules)):
        if len(molecules[n]) > longest:
            longest = len(molecules[n])
    return longest + 1

longest = find_longest_molecule(numeric_tokens)
print(f"Longest molecule: {longest}")

Longest molecule: 64


Nu is het tijd om de input en de targets voor het model te maken, dit bestaan uit input_tokens die de numeric token zijn maar dan met een extra 0 ervoor, dit is om het model te laten leren wat erna komt. Ook maak ik hier de target voor het model, dit zijn dan de numeric values maar met een extra 0 om de offset te houden en de lengte het zelfde te houden. Een RNN heeft dan ook de zelfde lengte input en output nodig.

Extra, het is ook een oplossing om een < END > en < START > token te maken om zo nog beter aan te geven wanneer het model moet starten en stoppen met de sequence. Dit kan ik dan in de vocabulary erbij zetten.

In [10]:
def shift_tokens(numeric_tokens, shift_num):
    shifted_tokens = []
    for molecule in numeric_tokens:
        molecule = [shift_num] + molecule    
        shifted_tokens.append(molecule)
    return shifted_tokens

def add_end_padding(numeric_tokens, padding_value):
    padded_tokens = []
    for molecule in numeric_tokens:
        molecule = molecule + [padding_value]
        padded_tokens.append(molecule)
    return padded_tokens



input_tokens = shift_tokens(numeric_tokens, 0)
target_tokens = add_end_padding(numeric_tokens, 0)
print(f"Input tokens: {input_tokens}")
print(f"Target tokens: {target_tokens}")

Input tokens: [[0, 10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16], [0, 10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12], [0, 12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4], [0, 10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4], [0, 10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 14, 14, 1, 10, 15, 2, 14, 1, 12, 10, 2, 14, 1, 10, 1, 8, 12, 2, 12, 10, 2, 14, 5, 2, 14, 14, 1, 10, 15, 2, 14, 4, 12, 10]]
Target tokens: [[10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16, 0], [10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8

Nu moet ik alleen nog zorgen dat alle inputs en targets daadwerkelijk de zelfde lengte hebben als het langste molecuul. Dit is heel makkelijk te doen.

In [ ]:
padding_num = 0



[[0, 10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16], [0, 10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12], [0, 12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4], [0, 10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4], [0, 10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 14, 14, 1, 10, 15, 2, 14, 1, 12, 10, 2, 14, 1, 10, 1, 8, 12, 2, 12, 10, 2, 14, 5, 2, 14, 14, 1, 10, 15, 2, 14, 4, 12, 10], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
